## Setting

In [3]:
#pip install langchain langchain-cohere langchain-groq langchain-community langchain-pinecone python-dotenv  pypdf

In [ ]:
# Importação de bibliotecas para lidar com variáveis de ambiente
import os
from dotenv import load_dotenv, find_dotenv

# Loaders para carregar arquivos de texto e PDF
from langchain_community.document_loaders import PyPDFLoader

# Ferramentas para dividir documentos em partes menores
from langchain_text_splitters import CharacterTextSplitter

# Geração de embeddings com Cohere
from langchain_cohere import CohereEmbeddings

# Integração com o vetorstore Pinecone
from langchain_pinecone import PineconeVectorStore

# Memória de conversa (não utilizada no trecho atual, mas importada)
from langchain.memory import ConversationBufferMemory

# LLM da Groq (ChatGPT acelerado por hardware especializado)
from langchain_groq import ChatGroq

# Acesso ao hub de chains pré-construídas do LangChain
from langchain import hub

# Chains para combinar documentos e fazer busca com recuperação
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

# Vectorstore local alternativo (FAISS), não utilizado nesse código
from langchain_community.vectorstores import FAISS

c:\Users\WINDOWS 10\RAG Retrieval-Augmented Generation\RAG-Retrieval-Augmented-Generation-Aprimorando-LLMs-com-Dados-Externos\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

Chaves API

In [57]:
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv(find_dotenv())

# Obtém as chaves de API das variáveis de ambiente
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [58]:
# Conferindo e pegou as Chaves corretamente
print(f"COHERE_API_KEY: {COHERE_API_KEY}")  # Debug temporário
print(f"PINECONE_API_KEY: {PINECONE_API_KEY}")  # Debug temporário
print(f"GROQ_API_KEY: {GROQ_API_KEY}")  # Debug temporário

COHERE_API_KEY: TOEQPo2gzPWKLP7SUpwAW3svbK3Vwm7tZOkU6FX5
PINECONE_API_KEY: pcsk_65d8mR_TRTfYVcecxyGRyRQPFNARgkRmit2WWY75Q4KVx3YFmmXHA3tiUKWZbsSzYsQSY5
GROQ_API_KEY: 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


Carregando e cortando arquivo

In [5]:
# Carrega o conteúdo do arquivo PDF
loader = PyPDFLoader('../data/2210.03629v3.pdf')
documents = loader.load()

In [60]:
# Divide o texto em partes menores para facilitar a vetorização
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000,chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Mostra quantas partes foram geradas
print(f"Total de partes criadas: {len(docs)}")

Total de partes criadas: 129


Incorporando agente

In [ ]:
# Instancia os embeddings usando o modelo da Cohere
embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
    cohere_api_key=COHERE_API_KEY
)

In [ ]:
# Nome do índice no Pinecone (deve já existir no console do Pinecone)
index_name = "pdf-demo"

# Cria o vetorstore a partir dos documentos, com os embeddings e configurações do Pinecone
vectorstore_from_docs = PineconeVectorStore.from_documents(
    docs,
    embedding=embeddings,
    index_name=index_name,
)

In [63]:
# Teste uma busca semântica:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
print(vectorstore.similarity_search("What is a vector store?"))

[Document(id='69385612-be28-4b2f-b6e9-6053f3bc33bc', metadata={'author': '', 'creationdate': '2023-03-13T00:09:11+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2023-03-13T00:09:11+00:00', 'page': 30.0, 'page_label': '31', 'producer': 'pdfTeX-1.40.21', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'source': 'data/2210.03629v3.pdf', 'subject': '', 'title': '', 'total_pages': 33.0, 'trapped': '/False'}, page_content='Published as a conference paper at ICLR 2023\nD.3 W EBSHOP TRAJECTORIES\nTable 10: Example trajectories for Webshop predicted by Act and ReAct. Compared to Act,\nReAct uses reasoning to ﬁnd products that satisfy all target attributes.\nInstruction: get me a sixteen pack of apple cinnamon freeze dried banana chips, and price lower than 50.00 dollars\nAct ReAct\nAction: search\n[\nsixteen pack apple cinnamon freeze dried banana\nchips\n]\nObservation:[\nBack to Search\n]\nPage 1 (Total results

Criando LLM e Chain

In [64]:
# Inicializa o LLM da Groq com Gemma2
llm = ChatGroq(
    model="Gemma2-9b-It",
    groq_api_key=GROQ_API_KEY,
    temperature=0.1
)                              

In [65]:
# Cria a chain de resposta com base nos documentos
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

# Cria o retriever para buscar documentos relevantes
retriever = vectorstore_from_docs.as_retriever()

# Cria a chain de RAG combinando busca + geração
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

c:\Users\WINDOWS 10\RAG Retrieval-Augmented Generation\RAG-Retrieval-Augmented-Generation-Aprimorando-LLMs-com-Dados-Externos\.venv\Lib\site-packages\langsmith\client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [66]:
# Testanto LLM 
res = rag_chain.invoke({"input":"O que é React? Responda em português."})
print("🔍 Pergunta: O que é React? Responda em português.")
print("💬 Resposta: \n")
print(res['answer'])

🔍 Pergunta: O que é React? Responda em português.
💬 Resposta: 

ReAct é um paradigma geral que combina raciocínio e ação com modelos de linguagem para resolver diversas tarefas de raciocínio e tomada de decisão em linguagem. 

Ele funciona promptando os modelos de linguagem a gerar tanto raciocínios verbais quanto ações relacionadas a uma tarefa de forma intercalada. Isso permite que o modelo realize raciocínio dinâmico para criar, manter e ajustar planos de alto nível para agir ("raciocinar para agir"), além de interagir com ambientes externos (como a Wikipedia) para incorporar informações adicionais ao raciocínio ("agir para raciocinar"). 


Em resumo, ReAct visa integrar a capacidade de pensar e agir em modelos de linguagem para solucionar problemas de forma mais completa e eficiente. 



## Finalização com interação com o usuário

In [ ]:
# Função interativa
def fazer_pergunta_rag():
    while True:
        pergunta = input("\n ❓ No que posso ajudar ❓ (ou digite 'sair' para encerrar):\n> ")
        
        if pergunta.lower() in ["sair", "exit", "quit"]:
            print("Encerrando. Até logo!")
            break

        try:
            resposta = rag_chain.invoke({"input":f"{pergunta} Responda em português"})
            print(f'🔍 Pergunta: {pergunta}')
            print("\n🧠 Resposta:\n")
            print(resposta["answer"])
        except Exception as e:
            print(f"\n⚠️ Erro ao gerar resposta: {e}")


In [68]:
# Execução
if __name__ == "__main__":
    fazer_pergunta_rag()

🔍 Pergunta: O que é react

🧠 Resposta:

O texto descreve um modelo chamado ReAct, que visa tornar as tarefas de resolução de problemas de grandes modelos de linguagem (LLMs) mais interpretáveis, diagnosticáveis e controlados por humanos. 

O ReAct conecta LLMs com um espaço de ações para interagir com ambientes externos, como a web ou ambientes físicos. No entanto, os autores reconhecem os potenciais perigos dessa conexão, como o acesso a informações privadas ou a execução de ações prejudiciais. 

Para minimizar esses riscos, os experimentos do ReAct se limitam a interações com sites específicos (Wikipedia ou WebShop) que não contêm informações privadas e sem ações perigosas no espaço de ações (os modelos não podem realmente comprar produtos no WebShop).

O texto também menciona que os autores fornecem todos os prompts usados, experimentos adicionais com GPT-3 e o código de prompting GPT-3 ReAct para aumentar a reprodutibilidade do estudo.


Em resumo, o ReAct é um modelo que tenta tor